In [177]:
import pandas as pd
import numpy as np
import os

In [178]:
os.chdir("/Users/rukhshan/Documents/GitHub/adv_ml/GenderProject")

In [179]:
df_uk = pd.read_excel("data/UK/uk_stanza_passive_900.xlsx")
df_uk['newspaper'] = ""
df_uk['newspaper'] = np.where(df_uk['link'].str.contains('thesun'), "sun", df_uk['newspaper'])
df_uk['newspaper'] = np.where(df_uk['link'].str.contains('thetimes'), "times", df_uk['newspaper'])
df_uk['newspaper'] = np.where(df_uk['link'].isnull(), "guardian", df_uk['newspaper'])
df_uk = df_uk[['title', 'text', 'combined_text', 'newspaper', 'passive_instances']]

df_mex = pd.read_excel("data/Mexico/mexico_stanza_passive_900_v2.xlsx")
df_mex['newspaper'] = ""
df_mex['newspaper'] = np.where(df_mex['link'].str.contains('heraldo'), "heraldo", df_mex['newspaper'])
df_mex['newspaper'] = np.where(df_mex['link'].str.contains('jornada'), "jornada", df_mex['newspaper'])
df_mex['newspaper'] = np.where(df_mex['link'].str.contains('eluni'), "eluni", df_mex['newspaper'])
df_mex = df_mex[['title', 'text', 'combined_text', 'newspaper', 'passive_instances']]

df_pak = pd.read_excel("data/Pakistan/pakistan_stanza_passive.xlsx")
df_pak = df_pak[['title', 'text', 'combined_text', 'newspaper', 'passive_instances']]


In [180]:
df_mex['num_passive'] = df_mex['passive_instances'].apply(lambda x: [x[1:3]])
df_mex['num_passive'] = df_mex['num_passive'].str[0].replace(",", "")

df_uk['num_passive'] = df_uk['passive_instances'].apply(lambda x: x[1:3])
df_uk['num_passive'] = pd.to_numeric(df_uk['num_passive'].str.replace(",", ""))

df_pak['num_passive'] = df_pak['passive_instances'].apply(lambda x: x[1:3])
df_pak['num_passive'] = pd.to_numeric(df_pak['num_passive'].str.replace(",", ""))


In [181]:
df_mex_table = df_mex.num_passive.value_counts().reset_index().rename(columns={'index': 'passive_instances', 'num_passive': 'frequency_of_articles'})
df_mex_table['passive_instances'] = pd.to_numeric(df_mex_table['passive_instances'].str.replace(",", ""))
df_mex_table = df_mex_table.sort_values(['passive_instances'])

df_mex_table_grp = df_mex.groupby('newspaper')['num_passive'].value_counts().reset_index(name='var').rename(columns={'num_passive': 'passive_instances', 'var': 'frequency_of_articles'})
df_mex_table_grp['passive_instances'] = pd.to_numeric(df_mex_table_grp['passive_instances'].str.replace(",", ""))
df_mex_table_grp = df_mex_table_grp.sort_values(['newspaper', 'passive_instances'])

df_mex_table.head()

,passive_instances,frequency_of_articles
0,0,327
2,1,122
1,2,127
3,3,88
4,4,57


In [182]:
df_pak_table = df_pak.num_passive.value_counts().reset_index().rename(columns={'index': 'passive_instances', 'num_passive': 'frequency_of_articles'})
df_pak_table = df_pak_table.sort_values(['passive_instances'])

df_pak_table_grp = df_pak.groupby('newspaper')['num_passive'].value_counts().reset_index(name='var').rename(columns={'num_passive': 'passive_instances', 'var': 'frequency_of_articles'})
df_pak_table_grp = df_pak_table_grp.sort_values(['newspaper', 'passive_instances'])

df_pak_table.head()



,passive_instances,frequency_of_articles
4,0,49
31,1,6
1,2,80
10,3,27
0,4,86


In [183]:
df_uk_table = df_uk.num_passive.value_counts().reset_index().rename(columns={'index': 'passive_instances', 'num_passive': 'frequency_of_articles'})
df_uk_table = df_uk_table.sort_values(['passive_instances'])

df_uk_table_grp = df_uk.groupby('newspaper')['num_passive'].value_counts().reset_index(name='var').rename(columns={'num_passive': 'passive_instances', 'var': 'frequency_of_articles'})
df_uk_table_grp = df_uk_table_grp.sort_values(['newspaper', 'passive_instances'])

df_uk_table.head()

df_uk_table_grp.head()



,newspaper,passive_instances,frequency_of_articles
30,guardian,0,3
22,guardian,2,6
38,guardian,3,1
5,guardian,4,12
23,guardian,5,6


In [184]:
#NOMALIZING


# Normalizing


In [185]:
df_mex['num_passive'] = pd.to_numeric(df_mex['num_passive'].str.replace(",", ""))

In [209]:
import spacy
from spacy.lang.es.examples import sentences 

nlp_es = spacy.load("es_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")


df_mex = df_mex[~df_mex['combined_text'].isnull()]
df_uk = df_uk[~df_uk['combined_text'].isnull()]
df_pak = df_pak[~df_pak['combined_text'].isnull()]

df_mex['tokenized_sents'] = df_mex['combined_text'].apply(lambda x: [sent.text for sent in nlp_es(x).sents])
df_uk['tokenized_sents'] = df_uk['combined_text'].apply(lambda x: [sent.text for sent in nlp_en(x).sents])
df_pak['tokenized_sents'] = df_pak['combined_text'].apply(lambda x: [sent.text for sent in nlp_en(x).sents])



In [224]:
df_mex['freq_sent'] = df_mex['tokenized_sents'].apply(lambda x: len(x))
df_uk['freq_sent'] = df_uk['tokenized_sents'].apply(lambda x: len(x))
df_pak['freq_sent'] = df_pak['tokenized_sents'].apply(lambda x: len(x))

In [238]:
df_mex['normalized'] = df_mex['num_passive'] / df_mex['freq_sent']
df_uk['normalized'] = df_uk['num_passive'] / df_uk['freq_sent'] / 2 #dividing by 2 because we are looking at a pattern
df_pak['normalized'] = df_pak['num_passive'] / df_pak['freq_sent'] /2 #dividing by 2 because we are looking at a pattern

In [241]:
df_mex.normalized.mean()

0.2515890322678212

In [242]:
df_pak.normalized.mean()

0.3139862897450189

In [243]:
df_uk.normalized.mean()

0.3103595054586266

In [244]:
df_mex.groupby('newspaper')['normalized'].mean()

newspaper
eluni      0.205411
heraldo    0.257259
jornada    0.256865
Name: normalized, dtype: float64

In [245]:
df_pak.groupby('newspaper')['normalized'].mean()

newspaper
dawn      0.315305
nation    0.293032
news      0.325382
Name: normalized, dtype: float64

In [246]:
df_uk.groupby('newspaper')['normalized'].mean()

newspaper
guardian    0.356950
sun         0.313266
times       0.258621
Name: normalized, dtype: float64

In [249]:
df_prelim = pd.read_csv("../data/data_labelling/NewsArticles.csv", encoding='')[['article_id', 'publish_date', 'article_source_link', 'title', 'text']]
df_prelim.head()

FileNotFoundError: [Errno 2] File b'../data/data_labelling/NewsArticles.csv' does not exist: b'../data/data_labelling/NewsArticles.csv'